In [112]:
import nibabel as nib
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [113]:
import os


def join_nii_files(file_name_pattern, parent_directory):


    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    individual_data = []

    for subdirectory in subdirectories:
        file_path = os.path.join(parent_directory, subdirectory, file_name_pattern)
        data_raw = nib.load(file_path)
        data = data_raw.get_fdata()
        data_flattened = data.flatten()  #convert 3d-array into a flattened 1d-array
        individual_data.append(data_flattened)
    
    return individual_data

joined_data = np.concatenate((join_nii_files("beta_0002.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Cont"),
                            join_nii_files("beta_0011.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Cont"),
                            join_nii_files("beta_0002.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Pct"),
                            join_nii_files("beta_0011.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Pct")), axis=0)


print(joined_data.shape)

brain_valid_data = np.nan_to_num(joined_data, nan=0)   #Replace all the NaN values with 0's

label = np.loadtxt("Data/K-Fold_PCL-5.txt", dtype="float")

(86, 173628)


In [114]:
print(brain_valid_data.shape)

(86, 173628)


In [115]:
def KFoldRegressor_TEPT(X, y):
    
    num_folds = 5
    num_samples = 86

    arr_ord = np.arange(num_samples)
    np.random.shuffle(arr_ord)

    arr_misc = np.concatenate((X[arr_ord[0:17], :], X[arr_ord[17:34], :], X[arr_ord[34:51], :], X[arr_ord[51:68], :], X[arr_ord[68:85], :]), axis = 0)
    label_misc = np.concatenate((y[arr_ord[0:17]], y[arr_ord[17:34]], y[arr_ord[34:51]], y[arr_ord[51:68]], y[arr_ord[68:85]] ), axis = 0)
    
    ind = [0, 17, 34, 51, 68, 85]


    arr_pred = []
    for i in range(5):
        
        rf = RandomForestRegressor(n_estimators = 50)

        slices_deleted = slice(ind[i], ind[i + 1])
        result_arr = np.concatenate((arr_misc[:slices_deleted.start], arr_misc[slices_deleted.stop:]))
        result_label = np.concatenate((label_misc[:slices_deleted.start], label_misc[slices_deleted.stop:]))


        rf.fit(result_arr, result_label)
        y_pred = rf.predict(arr_misc[ind[i]:ind[i+1]])

        arr_pred.append(y_pred)



    return arr_pred, label_misc

arr_pred, label_misc = KFoldRegressor_TEPT(brain_valid_data, label)


In [125]:
arr_total = np.concatenate((arr_pred[0], arr_pred[1], arr_pred[2], arr_pred[3], arr_pred[4]), axis = 0)
print(arr_total)

[14.94 29.62 36.58 20.12 25.76 34.08 24.06 23.94 22.2  26.48 30.74 38.14
 28.04 27.08 11.6  20.9  25.86 18.82 17.04 32.62 19.48 22.32 32.46 28.46
 26.72 34.4  37.38 39.22 32.3  26.06 18.5  24.78 26.62 34.68 26.62 33.16
 33.72 39.66 37.24 38.44 29.34 24.36 35.04 34.64 23.24 31.24 34.5  20.9
 25.86 32.68 22.1  29.84 20.1  17.46 17.24 33.1  28.92 24.3  23.6  40.82
 20.14 29.08 16.6  29.5  30.3  26.28 23.62 26.4  19.   27.94 25.64 31.7
 34.42 26.36 22.   28.26 15.44 11.3  26.58 19.92 31.54 30.58 24.66 15.46
 18.16]


In [126]:
import scipy

scipy.stats.pearsonr(arr_total, label_misc)

PearsonRResult(statistic=0.2506902954890549, pvalue=0.02066148410653251)